In [14]:
# -----------------------------
# 1. 패키지 임포트
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langgraph.graph import StateGraph, END
from typing import TypedDict, List
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
import re

# -----------------------------
# 2. 상태 정의
class CafeState(TypedDict):
    messages: List[BaseMessage]

# -----------------------------
# 3. 임베딩 + 벡터 DB 구성 (무료 모델 사용)
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

texts = [
    "아메리카노는 진한 에스프레소에 뜨거운 물을 더한 커피입니다. 가격은 ₩4,500입니다. 설명: 깔끔하고 부드러운 맛.",
    "카페라떼는 에스프레소에 따뜻한 우유를 넣은 커피입니다. 가격은 ₩5,000입니다. 설명: 부드럽고 고소한 풍미.",
    "바닐라라떼는 바닐라 시럽이 들어간 카페라떼입니다. 가격은 ₩5,500입니다. 설명: 달콤한 바닐라 향이 특징.",
]
menu_db = FAISS.from_texts(texts=texts, embedding=embedding)

# -----------------------------
# 4. 정보 추출 함수
def extract_menu_info(doc: Document) -> dict:
    content = doc.page_content
    price_match = re.search(r'₩([\d,]+)', content)
    description_match = re.search(r'설명:\s*(.+?)(?:\n|$)', content, re.DOTALL)
    return {
        "name": doc.metadata.get('menu_name', 'Unknown'),
        "price": price_match.group(0) if price_match else "가격 정보 없음",
        "description": description_match.group(1).strip() if description_match else "설명 없음"
    }

# -----------------------------
# 5. 문의 유형 분류 함수 (수정됨)
def classify_message(state: CafeState):
    user_msg = state["messages"][-1].content
    if "추천" in user_msg:
        return {"next": "recommend"}
    elif "가격" in user_msg:
        return {"next": "price"}
    elif "메뉴" in user_msg or "있어" in user_msg:
        return {"next": "menu"}
    else:
        return {"next": "fallback"}

# -----------------------------
# 6. 유형별 응답 함수
def handle_price(state: CafeState):
    docs = menu_db.similarity_search("메뉴 가격", k=3)
    response = "\n".join([doc.page_content for doc in docs])
    return {"messages": state["messages"] + [AIMessage(content=f"[가격 정보]\n{response}")]}

def handle_menu(state: CafeState):
    user_msg = state["messages"][-1].content
    docs = menu_db.similarity_search(user_msg, k=3)
    response = "\n".join([doc.page_content for doc in docs])
    return {"messages": state["messages"] + [AIMessage(content=f"[메뉴 정보]\n{response}")]}

def handle_recommend(state: CafeState):
    docs = menu_db.similarity_search("인기 메뉴", k=3)
    response = "\n".join([doc.page_content for doc in docs])
    return {"messages": state["messages"] + [AIMessage(content=f"[추천 메뉴]\n{response}")]}

def handle_fallback(state: CafeState):
    return {"messages": state["messages"] + [AIMessage(content="죄송합니다. 이해하지 못했어요. 메뉴, 가격, 추천에 대해 다시 말씀해 주세요.")]}

In [18]:
builder = StateGraph(CafeState)

# 일반 노드
builder.add_node("price", handle_price)
builder.add_node("menu", handle_menu)
builder.add_node("recommend", handle_recommend)
builder.add_node("fallback", handle_fallback)

# ✅ classifier는 반드시 노드로 등록
builder.add_node("classifier", classify_message)
builder.set_entry_point("classifier")

# ✅ source도 문자열로 지정
builder.add_conditional_edges(
    source="classifier",
    path=lambda x: x["next"],  # ✅ dict에서 'next' 값만 꺼냄!
    path_map={
        "recommend": "recommend",
        "menu": "menu",
        "price": "price",
        "fallback": "fallback",
    },
)

# ✅ 종료
builder.add_edge("price", END)
builder.add_edge("menu", END)
builder.add_edge("recommend", END)
builder.add_edge("fallback", END)

graph = builder.compile()


In [ ]:
# -----------------------------
# 8. 초기 상태 생성 및 실행
initial_state = CafeState(messages=[HumanMessage(content="메뉴 추천 좀 해줘")])
result = graph.invoke(initial_state)

# -----------------------------
# 9. 결과 출력
for msg in result["messages"]:
    print(msg.type, ":", msg.content)

human : 메뉴 추천 좀 해줘
ai : [추천 메뉴]
바닐라라떼는 바닐라 시럽이 들어간 카페라떼입니다. 가격은 ₩5,500입니다. 설명: 달콤한 바닐라 향이 특징.
아메리카노는 진한 에스프레소에 뜨거운 물을 더한 커피입니다. 가격은 ₩4,500입니다. 설명: 깔끔하고 부드러운 맛.
카페라떼는 에스프레소에 따뜻한 우유를 넣은 커피입니다. 가격은 ₩5,000입니다. 설명: 부드럽고 고소한 풍미.


: 